In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###model

In [ ]:



!pip install transformers
!huggingface-cli login
!pip install datasets
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer,pipeline
import transformers
import torch
import pandas as pd
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## subproblem

###data-process

In [ ]:
import pandas as pd

from datasets import load_dataset

# Load and select the first 100 examples
df1 = load_dataset("tasksource/Boardgame-QA")['train'].select(range(100)).to_pandas()
df = df1[['example']]  # Note the double brackets to keep the result as a DataFrame
df.head(2)

,example
0,A few players are playing a boardgame. The cur...
1,A few players are playing a boardgame. The cur...


In [ ]:
# Make a copy of the datasets
df_copy = df.copy()


### generate


In [ ]:
import pandas as pd
import torch
from datasets import load_dataset
# Function to generate CoT prompt
def generate_sub_response(complete_question):
    # Adjust the prompt directly with `complete_question` since it's already a string
    prompt = f"""Write a response that Break the input question into multiple subquestions based on the question provided.
                  Question:{complete_question}
                  Your Answer should follow the format:
                    subquestion1)
                    subquestion2)
                    subquestion3)
                    ......"""
    try:
        # Assuming the model and pipeline have been appropriately set up earlier
        response = pipeline(prompt, max_length=1500, num_return_sequences=1)
        # Extract the generated text from the response
        generated_text = response[0]['generated_text']
    except Exception as e:
        print(f"Error generating response: {e}")
        generated_text = "Error in generation"  # Use an empty string or a placeholder in case of an error

    return generated_text
generated_responses1 = []
count = 0
for index, row in df_copy.iterrows():
    # Extract the complete question and label from the current row
    complete_question = row['example']
    generated_response = generate_sub_response(complete_question)
    # Append the generated response to the list
    generated_responses1.append(generated_response)
    print("ID: ",count)
    count+=1
    # print(f"Generated Response: {generated_response}")
    # print("-----------------------------------------------------")

# df_copy.loc[:1, 'subproblem'] = generated_responses1
# # Save the augmented DataFrame to a new CSV file
# df_path_copy = 'df_copy.csv'
# df_copy.to_csv(df_path_copy, index=False)


In [ ]:
df_copy['subproblem'] = generated_responses1
# Save the augmented DataFrame to a new CSV file
df_path_copy = 'df_copy.csv'
df_copy.to_csv(df_path_copy, index=False)